In [3]:
cd /home

/home


In [4]:
import tomli 

from src.data import pretraining_pipeline
from src.pipeline import base
from src.models import get_ASTROMER_nsp

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%load_ext autoreload
%autoreload 2

In [5]:
config_file = './presentation/experiments/astromer_nsp/config_files/macho/alcock.100.f0.a.toml'
with open(config_file, mode="rb") as fp:
    config = tomli.load(fp)

In [6]:
d_model = config['astromer']['head_dim']*config['astromer']['heads']
astromer =  get_ASTROMER_nsp(num_layers=config['astromer']['layers'],
                             d_model=d_model,
                             num_heads=config['astromer']['heads'],
                             dff=config['astromer']['dff'],
                             base=config['positional']['base'],
                             dropout=config['astromer']['dropout'],
                             maxlen=config['astromer']['window_size'])

2022-11-07 13:08:30.688383: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 13:08:31.093178: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22349 MB memory:  -> device: 0, name: Graphics Device, pci bus id: 0000:82:00.0, compute capability: 8.6


In [7]:
astromer = base.compile_astromer(config, astromer, 
                      weights='./presentation/experiments/astromer_nsp/results/macho/pretraining/')

[INFO] Using custom scheduler
[INFO] Pretrained weights: ./presentation/experiments/astromer_nsp/results/macho/pretraining/


In [8]:
astromer.summary()

Model: "ASTROMER_NSP"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 203, 1)]     0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 203, 1)]     0           []                               
                                                                                                  
 times (InputLayer)             [(None, 203, 1)]     0           []                               
                                                                                                  
 encoder (Encoder)              (None, None, 256)    660736      ['input[0][0]',                  
                                                                  'mask[0][0]',        

In [13]:
data = pretraining_pipeline(os.path.join(config['pretraining']['data']['path'], 'test'),
                            config['pretraining']['data']['batch_size'],
                            config['astromer']['window_size'],
                            config['masking']['mask_frac'],
                            config['masking']['rnd_frac'],
                            config['masking']['same_frac'],
                            config['pretraining']['data']['sampling'],
                            config['pretraining']['data']['shuffle_test'],
                            repeat=1,
                            num_cls=None,
                            normalize=config['pretraining']['data']['normalize'],
                            cache=config['pretraining']['data']['cache_test'],
                            nsp_prob=.5,
                            nsp_frac=.5,
                            moving_window=False)

[INFO] Repeating dataset x1 times
[INFO] Using NSP
[INFO] Mov. win:  False


In [15]:
results = astromer.evaluate(data)

63/63 [==============================] - 101s 2s/step - loss: 0.4852 - rmse: 0.1590 - r_square: 0.7680 - bce: 0.3262 - acc: 0.9867
